In [ ]:
from datasets import load_dataset, load_metric
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
import torch
import numpy as np

In [ ]:
# 데이터셋 불러오기
ds = load_dataset("daekeun-ml/naver-news-summarization-ko")

# 데이터셋 확인
print(ds)

In [ ]:
# 데이터셋 준비
train_data = ds['train']
val_data = ds['validation']

# 토크나이저 로드
model_name = 'gogamza/kobart-summarization'  # KoBART 요약 모델
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

# 데이터 전처리
def preprocess_function(examples):
    inputs = tokenizer(examples['document'], max_length=1024, truncation=True, padding="max_length")
    targets = tokenizer(examples['summary'], max_length=128, truncation=True, padding="max_length")
    
    inputs["labels"] = targets["input_ids"]  # labels는 target summary의 input_ids
    return inputs

# 데이터셋 전처리
train_dataset = train_data.map(preprocess_function, batched=True)
val_dataset = val_data.map(preprocess_function, batched=True)

# 모델 로드
model = BartForConditionalGeneration.from_pretrained(model_name)

# Trainer 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 디렉토리
    evaluation_strategy="epoch",     # 평가 전략
    learning_rate=2e-5,              # 학습률
    per_device_train_batch_size=8,   # 학습 배치 사이즈
    per_device_eval_batch_size=8,    # 평가 배치 사이즈
    num_train_epochs=3,              # 학습 에폭 수
    weight_decay=0.01,               # 가중치 감소
)

trainer = Trainer(
    model=model,                         # 모델
    args=training_args,                  # 학습 인자
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=val_dataset,            # 평가 데이터셋
)

# 모델 학습
trainer.train()

# 모델 저장
model.save_pretrained("./fine_tuned_kobart")
tokenizer.save_pretrained("./fine_tuned_kobart")

# ROUGE metric 불러오기
rouge_metric = load_metric("rouge", trust_remote_code=True)

# 평가 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = np.where(labels != tokenizer.pad_token_id, labels, -100)  # pad token을 -100으로 설정
    pred_ids = torch.argmax(torch.tensor(logits), dim=-1)  # 예측값 생성
    predictions = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)  # 예측값 디코딩
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)  # 실제 값 디코딩
    
    # ROUGE 평가 지표 계산
    result = rouge_metric.compute(predictions=predictions, references=references)
    return result

# 평가 진행
trainer.compute_metrics = compute_metrics

# 모델 평가
eval_results = trainer.evaluate()

# 평가 결과 출력
print(eval_results)